In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 152
USE = 'clicks'

PART = 'B'

In [ ]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [ ]:
USR = np.load(f'test_user_{PART}.npy')
len( USR )

In [ ]:
LOAD = 709
OFFSET = ''

df = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df = df.loc[df.session_type.str.contains(USE)]
df = cudf.DataFrame(df)
print( df.shape )
df.head()

In [ ]:
OFFSET = '_wgt'

df2 = pd.read_csv(f"../../data/candidate_scores/submission_v{LOAD}{OFFSET}.csv")
df2 = df2.loc[df2.session_type.str.contains(USE)]
df2 = cudf.DataFrame(df2)
print( df2.shape )
df2.head()

In [ ]:
df['user'] = df.session_type.str.partition('_')[0].astype('int32')
df = df.loc[df.user.isin(USR)].reset_index(drop=True)
df = df.drop('session_type',axis=1)
df.head()

In [ ]:
df2['user'] = df2.session_type.str.partition('_')[0].astype('int32')
df2 = df2.loc[df2.user.isin(USR)].reset_index(drop=True)
df2 = df2.drop('session_type',axis=1)
df2.head()

In [ ]:
tmp = df.labels.str.split(' ').explode().astype('int32').rename('item').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

In [ ]:
users = df[['user']].copy()
users.head()

In [ ]:
users = users.merge(tmp, left_index=True, right_index=True)

In [ ]:
del df, tmp
gc.collect()

In [ ]:
users.head()

In [ ]:
users.dtypes

In [ ]:
tmp = df2.labels.str.split(' ').explode().astype('float32').rename('wgt').to_frame()
tmp['pos'] = np.tile( np.arange(100),tmp.shape[0]//100 ).astype('int32')
tmp.head()

In [ ]:
users2 = df2[['user']].copy()#.reset_index()
users2.head()

In [ ]:
del df2
gc.collect()

In [ ]:
users2 = users2.merge(tmp, left_index=True, right_index=True)

In [ ]:
users2.head()

In [ ]:
users = users.merge(users2,on=['user','pos'],how='left')

In [ ]:
users.head()

In [ ]:
del users2, tmp
gc.collect()

In [ ]:
users.shape

# Add Targets

In [ ]:
test_labels = cudf.read_parquet('../../data/train_data/test_labels.parquet')
test_labels = test_labels.loc[test_labels['type']==USE]
print( test_labels.shape )
test_labels.head()

In [ ]:
tmp = test_labels.ground_truth.explode().astype('int32').rename('item')
tmp.head()

In [ ]:
test_users = test_labels[['session']].astype('int32').copy()
test_users.columns = ['user']

In [ ]:
test_users = test_users.merge(tmp,left_index=True,right_index=True)

In [ ]:
del test_labels, tmp
gc.collect()

In [ ]:
test_users[USE] = np.int8(1)

In [ ]:
test_users.dtypes

In [ ]:
test_users.head()

In [ ]:
users = users.merge(test_users, on=['user','item'], how='left').fillna(0)

In [ ]:
del test_users
gc.collect()

In [ ]:
users.head()

In [ ]:
print(users.shape, users.dtypes)

In [ ]:
users = users.to_pandas()

# Save

In [ ]:
users.dtypes

In [ ]:
users[USE] = users[USE].astype('int8')

In [ ]:
users.to_parquet(f'../../data/train_with_features/train_v{VER}_{PART}.pqt',index=False)